In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install kaggle
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp "/content/drive/My Drive/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c shopee-product-detection-student
!unzip -q shopee-product-detection-student.zip

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout, Dense
from keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D
from keras.losses import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import warnings; 
import pandas as pd

warnings.filterwarnings("ignore")

IMAGE_SIZE = (200, 200)
INPUT_SHAPE = (200, 200, 3)
BATCH_SIZE = 128
SEED = 0

EPOCHS = 1
model_name = 'MobileNetV2_' + str(EPOCHS)
train_path = '/content/train/train/train'
test_path = '/content/test/test'

train_gen = ImageDataGenerator(rescale = 1./255, validation_split = 0.15)
train_set = train_gen.flow_from_directory(train_path, target_size = IMAGE_SIZE, \
                                          batch_size = BATCH_SIZE, seed = SEED, \
                                          subset = 'training')
val_set = train_gen.flow_from_directory(train_path, target_size = IMAGE_SIZE, \
                                        batch_size = BATCH_SIZE, seed = SEED, \
                                        subset = 'validation')

test_gen = ImageDataGenerator(rescale = 1./255)
test_set = test_gen.flow_from_directory(test_path, target_size = IMAGE_SIZE, \
                                        batch_size = BATCH_SIZE, seed = SEED)

class_num = len(train_set.class_indices)

final = get_model()
final.compile(optimizer= tf.keras.optimizers.Adam(0.003),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
hist = final.fit(train_set, epochs=EPOCHS, batch_size=BATCH_SIZE, \
                 validation_data=val_set, shuffle=False)

print(final.summary())

final.save('/content/drive/My Drive/Shopee/' + model_name + '.hdf5')
subm = pd.read_csv('/content/test.csv')
subm = subm.sort_values(by='filename', ignore_index=True)

fnames = sorted(os.listdir('/content/test/test/test'))
unbroken_index = np.where(np.vectorize(len)(np.array(fnames)) == 36)[0]
    
print(test_set)

y_pred = final.predict(test_set, batch_size=BATCH_SIZE)

pred = y_pred.argmax(axis=1)
pred = pred[unbroken_index]
subm['category'] = pred
subm['category'] = subm['category'].apply(lambda x : '%02d' % x) # zero pad
    
subm.to_csv('/content/submission.csv', index=False)


!kaggle competitions submit -c shopee-product-detection-student -f submission.csv -m $model_name

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

model = tf.keras.applications.MobileNetV2(
    input_shape=(32,32,3),
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=10,
    classifier_activation="softmax"
)


model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

print(ds_train)
print(ds_test)
history = model.fit(
    ds_train,
    epochs=100,
    validation_data=ds_test
)
print(history)

Shuffling and writing examples to /root/tensorflow_datasets/cifar100/3.0.2.incompleteMBGUDF/cifar100-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar100/3.0.2.incompleteMBGUDF/cifar100-test.tfrecord


Dataset cifar100 downloaded and prepared to /root/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


ValueError: ignored

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D

from tensorflow.keras import backend as K


def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def relu6(x):
    return K.relu(x, max_value=6.0)


def _conv_block(inputs, filters, kernel, strides):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    return Activation(relu6)(x)


def _bottleneck(inputs, filters, kernel, t, alpha, s, r=False):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    tchannel = K.int_shape(inputs)[channel_axis] * t
    cchannel = int(filters * alpha)

    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))

    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation(relu6)(x)

    x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if r:
        x = Add()([x, inputs])

    return x


def _inverted_residual_block(inputs, filters, kernel, t, alpha, strides, n):
    x = _bottleneck(inputs, filters, kernel, t, alpha, strides)

    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, alpha, 1, True)

    return x


def MobileNetv2(input_shape, k, alpha=1.0):
    inputs = Input(shape=input_shape)

    first_filters = _make_divisible(32 * alpha, 8)
    x = _conv_block(inputs, first_filters, (3, 3), strides=(2, 2))

    x = _inverted_residual_block(x, 16, (3, 3), t=1, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 24, (3, 3), t=6, alpha=alpha, strides=2, n=2)
    x = _inverted_residual_block(x, 32, (3, 3), t=6, alpha=alpha, strides=2, n=3)
    x = _inverted_residual_block(x, 64, (3, 3), t=6, alpha=alpha, strides=2, n=4)
    x = _inverted_residual_block(x, 96, (3, 3), t=6, alpha=alpha, strides=1, n=3)
    x = _inverted_residual_block(x, 160, (3, 3), t=6, alpha=alpha, strides=2, n=3)
    x = _inverted_residual_block(x, 320, (3, 3), t=6, alpha=alpha, strides=1, n=1)

    if alpha > 1.0:
        last_filters = _make_divisible(1280 * alpha, 8)
    else:
        last_filters = 1280

    x = _conv_block(x, last_filters, (1, 1), strides=(1, 1))
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, last_filters))(x)
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(k, (1, 1), padding='same')(x)

    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)

    model = Model(inputs, output)

    return model